In [1]:
import os
import sys

from mistralai import Mistral  #

In [ ]:
from dotenv import load_dotenv

module_path = os.path.abspath(os.path.join("../src"))
if module_path not in sys.path:
    sys.path.append(module_path)

load_dotenv()

In [ ]:
from unstructured_client import UnstructuredClient

unstructured_api_key = os.getenv("UNSTRUCTURED_API_KEY")
with UnstructuredClient(
    api_key_auth=unstructured_api_key,
    server_url="http://localhost:8800/general/v0/general",
) as client:
    res = client.general.partition(
        request={
            "partition_parameters": {
                "files": {
                    "content": open("../data/input/resume.docx", "rb"),
                    "file_name": "resume.docx",
                },
            },
        },
    )

print(res.elements[0]["metadata"]["text_as_html"])

resume = res.elements[0]["metadata"]["text_as_html"]


In [4]:
job_description = "Volkswagen Group Services is excited to present a fantastic opportunity to join our Volkswagen Digital:Hub in Spain. As part of the global Software Development Centre network in the Volkswagen Group, we constantly seek talented individuals ready to bring their unique skills to our projects.\nOur start-up environment is collaborative, innovative, and open to new ideas. We strive to cultivate a work culture that encourages individual growth, team empowerment, and project excellence. We value respect, teamwork, and the common purpose that binds us as a group. If you're ready to thrive in a dynamic start-up environment and be part of our family, supported by one of the largest companies in the world, we invite you to read further.\nAbout the Team and the Hub:Our culture is centered around fostering individual growth and team empowerment within a flexible and supportive environment.We prioritize respect, teamwork, and inclusivity, recognizing and valuing the unique contributions of each member.We embrace a work-life balance philosophy and encourage continuous personal and professional development.\nThe Role and Key Responsibilities:To anticipate, identify and eliminate blockers in the team.Ensure Guidelines, Best Practices and Architecture.Delivery Advocate & Enabler.Responsible of sharing the knowledge and the team agreements between the team.You will work on a day-to-day basis with our design, frontend, backend and testing team to build a scalable and solid product.Working using best practices (testing, code reviews, DDD, CI/CD, clean cose...) and Agile methodologies.Collaborate in the design of the cluster architecture.Create of new automated tests/alerts.Analysis of new functionalities/system improvements.Dev ops: Pipeline's maintenance, creation // Deployment configuration maintenance, creation.\nRequirements:BA Degree in Computer Sciences (or FP) and/or professional experience in Software.Software engineering background.At least 5 years of experience and 2 doing this role.Good management skills and team spirit.Experience with Java and Spring ecosystem, Spring boot, Go is a plus or another languages.Experience with testing tools like: Jest, Cypress, Testing library..Experience with AWS; AWS Glue, AWS Lambda, AWS CloudFormation Service, AWS Cloudwatch Service, AWS DynamoDB Service, AWS VPC (Amazon Virtual Private)...Experience with REST APIs.Experience with Azure Cloud.DevOps knowledge (Kubernetes, Azure DevOps)Fluent English and/or Spanish speakerFluent German speaker (nice to have)\nWhat we offer:Fully remote work capability, with an option to work from our office when needed.Access to professional development tools and free language courses.Flexible working hours to accommodate personal and professional needs.A competitive holiday package and access to a variety of employee discounts.\n\nJoin us at Volkswagen Digital:Hub to advance your career in a role where your administrative skills will directly contribute to the success and smooth operation of our innovative projects.\nFor more information on our data processing activities and your rights as a data subject, please consult our privacy policy.\nRef: Level C"

In [ ]:
import json

api_key = os.getenv("MISTRAL_API_KEY")
model = "mistral-large-latest"

mistral_client = Mistral(api_key=api_key)

chat_response = mistral_client.chat.complete(
    model=model,
    messages=[
        {
            "role": "system",
            "content": """You are a helpful assistant that helps user optimize their resume for a job description.
                    You will be given a CV and a job description and you have to do the following tasks:
                     - You have to update the resume so that it fits better to the position from the job description.
                     - Feel free to add new bullet points if necessary to the work experience, to better match the job description.
                     - Your output should be a JSON object with the following fields:
                        - "resume": the customized resume including the key changes made in markdown format.
                        - "changes": a list of the key changes made to the resume.
                    For this task, I will be paying a tip above 1000% if you do a good job.
                    My professional career fully depends on you, so please do your best.""",
        },
        {
            "role": "user",
            "content": f"""Here is the CV:
            <cv>
            {resume}
            </cv>
            Here is the job description:
            <job_description>
            {job_description}
            </job_description>""".format(
                resume=resume,
                job_description=job_description,
            ),
        },
    ],
    response_format={
        "type": "json_object",
    },
)

contents = chat_response.choices[0].message.content
print(type(contents))
dict_contents = json.loads(contents)

customized_resume = contents["resume"]
changes = contents["changes"]


In [ ]:
print(dict_contents)

In [38]:
from md2pdf.core import md2pdf

pdf = "../data/customized_resume.pdf"
md2pdf(
    pdf_file_path=pdf,
    md_content=customized_resume,
)